# Combine records when information is split across rows

Sometimes data for the same record is split across subsequent rows. This is particularly the case when you extract data from a PDF table using a tool like Tabula.

In [1]:
# Setup
import numpy as np
import pandas as pd

/Users/ghing/.local/share/virtualenvs/python-data-cheatsheet-nEkgY1LY/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/ghing/.local/share/virtualenvs/python-data-cheatsheet-nEkgY1LY/lib/python3.6/site-packages/pandas/compat/__init__.py:84: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Create some sample data

In this data, it seems like the `message` value for the record with index 1 continues in the next row. However, there is no common identifier that can tie these rows together.

In [2]:
raw_data = [
    ('1/2/2019', 'Facility 1', "A log message."),
    ('1/2/2019', 'Facility 1', "A longer log message "),
    (None, None, "that continues onto a second row."),
    ('1/3/2019', 'Facility 3', "Another log message."),
    
]
columns = ['date', 'facility', 'message']

data = pd.DataFrame.from_records(raw_data, columns=columns)

data

,date,facility,message
0,1/2/2019,Facility 1,A log message.
1,1/2/2019,Facility 1,A longer log message
2,None,None,that continues onto a second row.
3,1/3/2019,Facility 3,Another log message.


## Add an ID

We'll start by adding an arbitrary ID to each row. We can just use the index.

In [3]:
data['id'] = data.index

data

,date,facility,message,id
0,1/2/2019,Facility 1,A log message.,0
1,1/2/2019,Facility 1,A longer log message,1
2,None,None,that continues onto a second row.,2
3,1/3/2019,Facility 3,Another log message.,3


## Remove the ID for rows that continue previous records

In [4]:
data['id'] = np.where(pd.isna(data['date']), None, data['id'])

data

,date,facility,message,id
0,1/2/2019,Facility 1,A log message.,0
1,1/2/2019,Facility 1,A longer log message,1
2,None,None,that continues onto a second row.,None
3,1/3/2019,Facility 3,Another log message.,3


## Fill the now missing IDs with the ID of the previous record

After doing this, the continued record will share an ID with the row that has most of the rest of the information.

In [5]:
data['id'] = data['id'].fillna(method='ffill')

data

,date,facility,message,id
0,1/2/2019,Facility 1,A log message.,0
1,1/2/2019,Facility 1,A longer log message,1
2,None,None,that continues onto a second row.,1
3,1/3/2019,Facility 3,Another log message.,3


## Use grouping and aggregation to merge the rows

We'll group by the synthetic ID that we created and then use the [form of the agg method that lets us specify separate aggregation functions for each column](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#applying-different-functions-to-dataframe-columns). I'm not entirely sure what the type of the data is that gets passed to the functions that are the values of the dictionary, but I think they're `Series`.

For the columns that don't continue on multiple rows, we use a lambda function that returns the value from the first row in the group. For the `message` column that is continued, we use the expression `' '.join`.

In [6]:
data = data.groupby('id')\
    .agg({
        'date': lambda x: x.iloc[0],
        'facility': lambda x: x.iloc[0],
        'message': ' '.join,
    })

data

,date,facility,message
id,,,
0,1/2/2019,Facility 1,A log message.
1,1/2/2019,Facility 1,A longer log message that continues onto a se...
3,1/3/2019,Facility 3,Another log message.


## Remove the temporary ID we used for grouping

In [7]:
data = data.reset_index(drop=True)

data

,date,facility,message
0,1/2/2019,Facility 1,A log message.
1,1/2/2019,Facility 1,A longer log message that continues onto a se...
2,1/3/2019,Facility 3,Another log message.
